# Blah

In [13]:
from IPython.display import display, HTML, Image , Markdown
from snowflake.snowpark.session import Session
from snowflake.snowpark.types import * 
from snowflake.snowpark.functions import *
import os ,configparser ,json

# Source various helper functions
%run ../scripts/notebook_helpers.py

PROJECT_HOME_DIR = '../../'
CONFIG_FL = f'{PROJECT_HOME_DIR}/config.ini'

In [14]:
display(Markdown("### Initialization"))

config = configparser.ConfigParser()
sp_session = None

print(" Initialize Snowpark session")
with open(CONFIG_FL) as f:
    config.read(CONFIG_FL)
    snow_conn_flpath =  f"{PROJECT_HOME_DIR}/{config['APP']['connection_fl']}"
    
    # ------------
    # Connect to snowflake
    with open(snow_conn_flpath) as conn_f:
        snow_conn_info = json.load(conn_f)
        sp_session = Session.builder.configs(snow_conn_info).create()

if(sp_session == None):
    raise(f'Unable to connect to snowflake. Validate connection information in file: {CONFIG_FL} ')

sp_session.sql(f''' use role {config['APP']['role']}; ''').collect()
sp_session.sql(f''' use database {config['APP']['database']}; ''').collect()
sp_session.sql(f''' use warehouse {config['APP']['warehouse']}; ''').collect()

df = sp_session.sql('select current_account() ,current_user() ,current_role();').to_pandas()
display(df)

### Initialization

 Initialize Snowpark session


,CURRENT_ACCOUNT(),CURRENT_USER(),CURRENT_ROLE()
0,ANA95816,VSEKAR,DEV_PCTRANSPERANCY_DEMO_RL


In [15]:
stmts = [
    f''' create or replace stage {config['APP']['database']}.public.data_parsed_stg
            directory = (enable=true)
            comment = 'used for holding parsed record.'; '''
]
for stmt in stmts:
    sp_session.sql(stmt).collect()

In [16]:
data_file = '2022_10_01_priority_health_HMO_in-network-rates.zip'
stage_path = 'data_stg/price_transperancy'

# data_file = '2022_07_01_priority_health_HMO_in-network-rates.json'
# data_file = '2022-11-01_cigna-health-life-insurance-company_national-oap_in-network-rates.json.gz'
# stage_path = 'ext_data_stg/data'

In [19]:
# Upload libraries to stage
lib_stage = config['APP']['lib_stage']

upload_locallibraries_to_p_stage(sp_session ,'../python/in-network-rates_v2' ,config['APP']['database'] ,'public' ,lib_stage ,'scripts')

print(' List stage directory !!')
stage_spdf = sp_session.sql(f'''list @{config['APP']['database']}.public.{lib_stage}; ''').collect()

for r in stage_spdf:
    print(f'@{r["name"]}')


 Uploading library to stage: sflk_pricing_transperancy.public.lib_stg 
    ../python/in-network-rates_v2/ingest-in-network-rates-seg-header_sp_v2.py => @lib_stg/scripts
    ../python/in-network-rates_v2/in-network-rates-segment-dagbuilder-v2-segment-count.py => @lib_stg/scripts
    ../python/in-network-rates_v2/ingest-in-network-rates-seg-header_sp_v2.py.orig => @lib_stg/scripts
    ../python/in-network-rates_v2/ingest-in-network-rates-segment-lists_sp.py => @lib_stg/scripts
    ../python/in-network-rates_v2/in-network-rates-segment-dagbuilder-v2.py => @lib_stg/scripts
 List stage directory !!
@lib_stg/scripts/in-network-rates-segment-dagbuilder-v2-segment-count.py
@lib_stg/scripts/in-network-rates-segment-dagbuilder-v2.py
@lib_stg/scripts/ingest-in-network-rates-seg-header_sp_v2.py
@lib_stg/scripts/ingest-in-network-rates-seg-header_sp_v2.py.orig
@lib_stg/scripts/ingest-in-network-rates-segment-lists_sp.py


In [20]:
from datetime import datetime
now = datetime.now()

start_time = now.strftime("%H:%M:%S")
print(f'Started at: {start_time}')

sql_stmts = [
    f''' alter warehouse dev_pctransperancy_demo_wh set warehouse_size = MEDIUM; '''
    ,f'''truncate table {config['APP']['database']}.public.in_network_rates_segment_header_V2; '''
    ,f''' call {config['APP']['database']}.public.innetwork_rates_segheader(
            1500000 ,'{stage_path}' ,'{data_file}');'''
]
for stmt in sql_stmts:
    sp_session.sql(stmt).collect()

now = datetime.now()
end_time = now.strftime("%H:%M:%S")
print(f'Ended at: {end_time}')

Started at: 17:23:36
Ended at: 18:16:38


---
### TBL

In [9]:
df = sp_session.table(f'''{config['APP']['database']}.public.in_network_rates_segment_header_V2''').limit(10).to_pandas()
display(df)

,SEQ_NO,DATA_FILE,SEGMENT_ID,NEGOTIATED_RATES_NAME,NEGOTIATED_RATES_INFO,NEGOTIATED_RATES_COUNT,BUNDLED_CODES_COUNT,COVERED_SERVICES_COUNT,INSERTED_AT
0,NaN,2022_10_01_priority_health_HMO_in-network-rate...,c641d820-b3c5-4f43-baef-db10612422d7,"Injection,Lumbar Epidural,Of Bloodor Cl","{'negotiation_arrangement': 'ffs', 'name': 'In...",5859,-1,-1,2022-11-18 20:28:08.480
1,NaN,2022_10_01_priority_health_HMO_in-network-rate...,30b61c61-3d83-41af-9ff4-2d1af9f69b59,SEIZURE AGE >17 W/O CC,"{'negotiation_arrangement': 'ffs', 'name': 'SE...",3,-1,-1,2022-11-18 20:28:08.480
2,NaN,2022_10_01_priority_health_HMO_in-network-rate...,dd1e1d18-d931-491e-a347-2e9d147b0f54,Infectious agent detection by nucleic acid (DN...,"{'negotiation_arrangement': 'ffs', 'name': 'In...",7569,-1,-1,2022-11-18 20:28:08.480
3,NaN,2022_10_01_priority_health_HMO_in-network-rate...,da033bfb-98d2-48c5-970f-327f0b89371e,"Ldh,Blood;isoenzymes,Electrophoretic Sep","{'negotiation_arrangement': 'ffs', 'name': 'Ld...",26,-1,-1,2022-11-18 20:28:08.480
4,NaN,2022_10_01_priority_health_HMO_in-network-rate...,b7a5e2c8-ec89-4376-9d2d-cd76f4edc8c8,"Thoracoscopy, Surgical; W Total Pulmonary Deco...","{'negotiation_arrangement': 'ffs', 'name': 'Th...",4035,-1,-1,2022-11-18 20:28:08.480
5,NaN,2022_10_01_priority_health_HMO_in-network-rate...,4c130ce3-f2ff-4464-a98d-5775a9f0c664,Close Enterostom Resect & Anastomos,"{'negotiation_arrangement': 'ffs', 'name': 'Cl...",9210,-1,-1,2022-11-18 20:28:08.480
6,NaN,2022_10_01_priority_health_HMO_in-network-rate...,a66e1815-ac44-40b6-9134-d13faa4bc58f,Antibody Cytomegalovirus (Cmv) Igm,"{'negotiation_arrangement': 'ffs', 'name': 'An...",81,-1,-1,2022-11-18 20:28:08.480
7,NaN,2022_10_01_priority_health_HMO_in-network-rate...,71b09ade-5a60-466d-ab8f-d718c4b76664,"CIRCULATORY DISORDERS EXCEPT AMI, WITH CARDIAC...","{'negotiation_arrangement': 'ffs', 'name': 'CI...",3,-1,-1,2022-11-18 20:28:08.480
8,NaN,2022_10_01_priority_health_HMO_in-network-rate...,6891ff3e-2008-4c67-aa47-7dc264ca14cb,"TESTES PROCEDURES, NON-MALIG AGE 0-17","{'negotiation_arrangement': 'ffs', 'name': 'TE...",3,-1,-1,2022-11-18 20:28:08.480
9,NaN,2022_10_01_priority_health_HMO_in-network-rate...,4e7f8232-a287-408c-96e0-28f12d0416b3,"Sensit.Stdy,Antibi;disk Meth,Ea.Plt","{'negotiation_arrangement': 'ffs', 'name': 'Se...",70,-1,-1,2022-11-18 20:28:08.480


#### TODO : Perform some model analysis 
 - accuracy score
 - error rate calculation
 - confusion matrix
  ...

--- 
### Closeout

    With that we are finished this section of the demo setup

In [10]:
# sp_session.close()
print('Finished!!!')

Finished!!!
